In [1]:
import numpy as np
import pandas as pd
import qgrid
from ipywidgets import interact, interactive, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [2]:
def minCostToThreshold(data, threshold):
    memo = {}

    def minCostFrom(index, remaining):
        if (index, remaining) in memo:
            return memo[(index, remaining)]

        if remaining <= 0:
            # Base case: no threshold remaining, succeed at zero cost with no contributing items"
            return (0, [])
        
        if index == len(data):
            # Base case: positive threshold and no items remaining, fail"
            return None
        
        item = data.iloc[index, :]
        _withItem = minCostFrom(index + 1, remaining - item.at['EV'])
        withItem = None if _withItem == None else (_withItem[0] + abs(item['Margin']), _withItem[1] + [index])
        withoutItem = minCostFrom(index + 1, remaining)
        ret = withItem if withoutItem == None or (withItem != None and withItem[0] < withoutItem[0]) else withoutItem

        memo[(index, remaining)] = ret
        return ret
    
    return minCostFrom(0, threshold)

def minVotesToElectoralMargin(stateData, evMargin):
    if evMargin == 0:
        return 0 # No margin
    elif any((stateData == 0).values.flatten()):
        raise "All inputs must be non-zero"
    elif evMargin > 0:
        flippableStates = stateData['Margin'] > 0
    elif evMargin < 0:
        flippableStates = stateData['Margin'] < 0
    
    flippableView = stateData[flippableStates]
    result = minCostToThreshold(flippableView, abs(evMargin))
    if result == None:
        return None
    else:
        votes, indicies = result
        return (votes, flippableView.iloc[indicies, :])

In [3]:
stateData2016 = [
    ( 9,   588708, 'Ala'),
    ( 3,    46933, 'Alaska'),
    (11,    91234, 'Ariz.'),
    ( 6,   304378, 'Ark.'),
    (55, -4269978, 'Calif.'),
    ( 9,  -136386, 'Colo.'),
    ( 7,  -224357, 'Conn.'),
    ( 3,   -50476, 'Del.'),
    ( 3,  -270107, 'D.C'),
    (29,   112911, 'Fla.'),
    (16,   211141, 'Ga.'),
    ( 4,  -138044, 'Hawaii'),
    ( 4,   219290, 'Idaho'),
    (20,  -944714, 'Ill.'),
    (11,   524160, 'Ind.'),
    ( 6,   147314, 'Iowa'),
    ( 6,   244013, 'Kan.'),
    ( 8,   574177, 'Ky.'),
    ( 8,   398484, 'La.'),
    ( 2,   -22142, 'Maine'),
    ( 1,   -58390, 'ME-1'),
    ( 1,    36360, 'ME-2'),
    (10,  -734759, 'Md.'),
    (11,  -904303, 'Mass.'),
    (16,    10704, 'Mich.'),
    (10,   -44765, 'Minnesota'),
    ( 6,   215583, 'Mississippi'),
    (10,   523443, 'Missouri'),
    ( 3,   101531, 'Montana'),
    ( 2,   211467, 'Nebr.'),
    ( 1,    58500, 'NE-1'),
    ( 1,     6534, 'NE-2'),
    ( 1,   146367, 'NE-3'),
    ( 6,   -27202, 'Nev.'),
    ( 4,    -2736, 'N.H.'),
    (14,  -546345, 'N.J.'),
    ( 5,   -65567, 'N.M.'),
    (29, -1736590, 'N.Y.'),
    (15,   173315, 'N.C.'),
    ( 3,   123036, 'N.D.'),
    (18,   446841, 'Ohio'),
    ( 7,   528761, 'Okla.'),
    ( 7,  -219703, 'Ore.'),
    (20,    44292, 'Pa.'),
    ( 4,   -71982, 'R.I.'),
    ( 9,   300016, 'S.C.'),
    ( 3,   110263, 'S.D.'),
    (11,   652230, 'Tenn.'),
    (38,   807179, 'Texas'),
    ( 6,   204555, 'Utah'),
    ( 3,   -83204, 'Vt.'),
    (13,  -212030, 'Va.'),
    (12,  -520971, 'Wash.'),
    ( 5,   300577, 'W.Va.'),
    (10,    22748, 'Wis.'),
    ( 3,   118446, 'Wyo.'),
]
evMargin2016 = 38

dframe2016 = pd.DataFrame(stateData2016, columns=['EV', 'Margin', 'State'])
qgrid_widget = qgrid.show_grid(dframe2016, show_toolbar=True)
display(qgrid_widget)

marginWidget = widgets.IntText(
    value=evMargin2016,
    description='EV Margin',
    disabled=False
)
display(marginWidget)

button = widgets.Button(description="Calculate")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        df = qgrid_widget.get_changed_df()
        votes, states = minVotesToElectoralMargin(df, marginWidget.value)
        result = states.copy()
        result.loc['Total'] = [sum(result['EV']), sum(result['Margin']), '-']
        display(widgets.Label(value=f'Minimum votes to flip election: {votes}'))
        display(qgrid.show_grid(result, show_toolbar=True))
    

button.on_click(on_button_clicked)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

IntText(value=38, description='EV Margin')

Button(description='Calculate', style=ButtonStyle())

Output()